**reference**

https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study            
https://datascience.stackexchange.com/questions/9443/when-to-use-one-hot-encoding-vs-labelencoder-vs-dictvectorizor
https://zzsza.github.io/data/2018/09/08/feature-engineering/


# Encodings

대부분의 머신러닝 알고리즘(Decision tree, Random forest 등은 제외)은 numerical values만을 input으로 받아들이므로, categorical value들은 encoding을 통해 변형시켜줘야한다.

porto-seguro-safe-driver-prediction competition(https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) 을 수행하며 categorical feature들의 cardinality를 확인한 결과, 아래와 같이 ps_car_11 feature가 100개가 넘는 cardinality를 가진 것을 확인했다.

In [ ]:
import pandas as pd

train = pd.read_csv('../input/train.csv', nrows=None)
train = train.sample(frac=0.2)

#categorical한 데이터들이 정말 integer로 되어있는지 확인
cat_cols = [col for col in train.columns if 'cat' in col]
# train[cat_cols[0]].value_counts()

#categorical data들의 카테고리 수 보기
for col in cat_cols:
    print(col, train[col].nunique())

이러한 데이터를 encoding방법 중 하나인 OHE(Onehot encoding)한다면 104개를 가진 변수때문에 column수가 많이 증가하게 되어 연상량이 증가하고, tree모델 사용 시 정답으로 가기위한 과정이 멀어진다는 단점이 있다(원하지 않는 split이 생길 가능성). 따라서 이러한 경우에는 mean encoding, frequency encoding 등이 많이 사용된다.

encoding방법에 따라 모델의 성능이 좌지우지되므로 이러한 encoding방법 문제는 짚어볼 필요가 있으며, 따라서 각 encoding 방법은 무엇인지, 언제 그 encoding방법을 사용하는 것이 좋을지 등을 알아보고자 한다.

특히 tree기반의 모델인 경우에는 categorical 변수를 encoding할 때, frequency, mean, label encoding 등을 사용함. 하지만 label encoding의 경우 타겟값과 상관없는 값이 코딩되는 것이기 때문에 성능이 안좋을 수 있다.



categorical feature를 제거하냐 안하냐를 선택할 때는 주로 feature에 따른 타겟값의 분포를 보고 선택함. 카테고리에 따른 타겟값의 분포가 차이가 없다면 그 feature는 없어도 무방함.

# One-Hot Encoding

OHE는 결과가 categorical이 아닌 binary이며, 따라서 모든 것이 직교 벡터 공간에 놓여진다는 특징을 가진다. 따라서 nominal feature에 사용하기 좋다.
Tree model보다는 linear models and SVMs에서 유용하다. 또한 feature와 target의 의존성이 non-linear하면 유용하게 사용이 가능하다.
neural network기반의 모델의 경우 주로 one-hot encoding을 이용한다. 다만 OHE를 이용할 경우 차원이 커지는 문제가 존재한다(차원의 저주). 이를 해결하기 위해 일반적으로 OHE는 뒤에 PCA 과정을 더해 차원감소과정을 추가하여 사용한다. 하지만 PCA는 linear overlap을 찾기때문에 비슷한 feature는 동일한 feature로 그룹화하기 때문에 성능이 그렇게 좋지는 않다.

![](https://cdn-images-1.medium.com/max/1600/0*T5jaa2othYfXZX9W.)

장점
* 실행하기 편하다
* 0 아니면 1로 값이 정해지기 때문에 category들이 확실하게 분리가 된다. 또한 category와 encoding 방법의 관계가 가정되지 않기때문에 model이 덜 bias될 것이다.

단점
* high cardinality feature의 경우, OHE를 이용하면 column의 수가 굉장히 많아짐. 이로 인해 연산이 많아지게 된다는 단점이 있으며 OHE된 변수를 중요한 변수라고 착각할 수가 있음(OHE로 인해 한 변수가 여러 column을 만들게 되므로 tree model에서 feature들을 랜덤하게 sampling할 때 OHE된 feature가 선택될 가능성이 높아지기 때문).
* tree model은 split을 하며 내려가는데, one-hot을 이용하면 값이 0 아니면 1이기 때문에 이 사이를 split할 방법이 없음. 따라서 tree는 좀 더 많은 split과정을 가져야하므로 학습이 느려지고, tree의 quality가 떨어지게 됨.

# Label encoding

label encoding은 각 카테고리 값을 숫자나 label로 mapping해주는 encoding 방법을 말한다. 예를들어, [Doctor, Engineer, Teacher, Doctor]을 [1, 2, 3, 1]로 바꿔준다.
categorical 변수를 사용할 수 있는 DT와 RF에서는 label encoding을 통해 적은 disk공간을 사용하여 값을 저장할 수 있는 장점이 있다. 또한 feature와 target의 의존성이 linear하면 유용하게 사용이 가능하다. 하지만, label encoding을 이용하면 의사와 선생님의 평균이 엔지니어가 된다는 것을 의미한다는 단점이 있다. 따라서 nominal feature가 아닌 경우에 사용하는 것이 적절하다. 

# Frequency encoding

특정 카테고리의 빈도를 label로 사용하는 방법을 말한다(Target으로 그 label값이 전체 데이터 셋 중 몇 %인지). 각 label의 빈도가 Target값과 관련이 있는 경우에 유용하다.

**OHE와 비교한 Label & frequency encoding 의**

장점
* 학습속도가 빠름.
* split시 한 번에 여러 카테고리를 분리할 수 있으므로(OHE는 1아니면 0이라 불가능), split을 더 적게해도 되서 더 robust한 model이 가능함.


단점
bias가 존재함. label robust(강력한 모델)

# Mean Encoding

label encoding의 변형으로, 각 카테고리 별 target값(1)의 비율을 label로 사용하는 방법을 말한다. 주의할 점은, test data의 target값은 제외하고 수행해야한다는 점이다. 
high cardinality feature를 가졌을 때 많이 사용되며, 특히 트리 기반 모델에서 많이 쓰임.

feature와 target값이 상관관계를 갖도록 하면서 차원문제도 해결할 수 있기 때문에 많이 사용함.

![](https://www.dropbox.com/s/9yd335yu9qg4ovu/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202018-08-23%2019.35.51.png?raw=1)

label encoding과 비교
장점
* split이 적어지고, 더 빠르게 학습이 가능함. 왜냐하면 tree는 label과 target값 간의 선형관계를 이용할 수 있고, 이것은 high cardinality에서 특히 유용하기 때문이다. (Fewer splits, faster learning. Trees can utilize the linear relationships between labels and target. This is especially useful when working with high cardinality categorical features: it is hard for a model to put every small category into a separate bucket, but if a lot of small categories can be put in one bin together based on their mean target value, then trees can learn way faster.)
* bias가 적어진다. label이 좀 더 의미를 갖기때문이다. 아래 그림을 보면 mean encoding된 label값들이 해당 label값에 몰려있는 것을 알 수 있음. Label encoding은 target에 대한 correlation이 없고 mean encoding은 0과 1로 나뉘는 형태를 띄며 Target값과 직접적 연관성이 생김)

단점
* 구축하는 것과 검증하는 것이 어려움
* target값과 직접적으로 연관시키기 때문에 overfitting되기 쉬우므로 regularization과정이 필요함

![](https://www.dropbox.com/s/2tukemvxhwndp5t/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202018-08-23%2019.45.13.png?raw=1)

overfitting이 심하므로(target값을 기준으로 encoding한거기 때문에) 이를 해결하기 위해 regularization이 필요하며, 크게 4개가 있음.
1. Training data에서 Cross-validation loop돌리기
2. Smoothing
3. Random noise 추가
4. Sorting and calculating expanding mean

1. CV loop 돌리기

가장 강력한(robust) 방법이며, 직관적이다. 보통 4~5fold를 진행하면 괜찮은 결과가 나온다고 한다. train data에 없고 validation에만 있는 category가 있을 수 있는데, 이런 경우에는 NA값을 global mean값으로 설정해줌.

LOO(leave one out)같은 극단적 상황을 조심해야 함.

2. smoothing

hyperparameter인 alpha값을 사용하여 regularization을 진행함. 보통 alpha값이 카테고리 사이즈로 설정됨.

![](https://www.dropbox.com/s/xjuj2kah7evaipn/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202018-08-23%2020.32.42.png?raw=1)

3. Rancom noise 추가

mean encoding한 값에 noise를 추가하는 방법을 말함. 하지만 encoding의 quality를 저하하고, 얼마나 noise를 추가해줘야하는지 등의 고민이 필요하기 때문에 불안정하여 잘 사용하지 않음. 보통 LOO와 같이 사용됨

4. expanding mean

target variable에서 leakage가 최소화되고, hyperparameter가 없으므로 고민할게 없어서 편함. 또한 catboost에 built-in 되어 있어 사용하기 편하고, 사용 시 많은 성능 향상을 가져옴. 하지만 단점은 encoding의 quality가 불규칙적이라는 것이다.

![](https://www.dropbox.com/s/rfjc4c1dzegwfzu/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202018-08-23%2020.35.36.png?raw=1)